In [13]:
from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
#API#1
os.environ["GROQ_API_KEY"] = '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'
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],

    model_name="llama-3.3-70b-versatile",       

    temperature=0.0,
  #  max_tokens=1000,
)

In [14]:
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_openai import ChatOpenAI

def preprocess_sparql(query: str) -> str:
    query = re.sub(r'SERVICE\s+wikibase:label\s*\{[^}]*\}', '', query, flags=re.IGNORECASE)
    if 'rdfs:label' not in query:
        query += '\nOPTIONAL { ?s rdfs:label ?label . FILTER(lang(?label) = "zh") }'
    if 'SELECT' not in query.upper():
        query = f"SELECT * WHERE {{ {query} }}"
    return query.strip()

def query_kg_tool(sparql_query: str) -> str:
    endpoint = "http://192.168.133.39:8890/sparql"
    headers = {"Accept": "application/sparql-results+json"}
    params = {"query": sparql_query}
    try:
        response = requests.get(endpoint, headers=headers, params=params, timeout=10)
        response.raise_for_status()

        if "html" in response.headers["Content-Type"]:
            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.find_all('tr')[1:]
            output = []
            for row in rows:
                cols = row.find_all('td')
                output.append(f"Subject: {cols[0].get_text()}, Predicate: {cols[1].get_text()}, Object: {cols[2].get_text()}")
            return "\n".join(output)

        results = response.json()
        bindings = results.get("results", {}).get("bindings", [])
        if not bindings:
            return "查無資料。"

        output = []
        for row in bindings:
            row_data = {k: v.get("value") for k, v in row.items()}
            output.append(str(row_data))
        return "\n".join(output)
    except Exception as e:
        return f"查詢失敗：{str(e)}"

# 工具註冊
local_sparql_tool = Tool(
    name="本地 SPARQL 查詢工具",
    func=query_kg_tool,
    description="使用 SPARQL 查詢本地知識圖譜"
)



# 初始化 Agent
agent = initialize_agent(
    tools=[local_sparql_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [15]:
# 查詢語句：請查詢 '李登輝' 的相關資訊
query = "查詢與'電影'有關的文件"

# 使用 agent 查詢本地知識圖譜
response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
Thought: 我需要查詢與"電影"有關的文件，所以我應該使用SPARQL查詢工具來查詢本地知識圖譜。

Action: 本地 SPARQL 查詢工具
Action Input: PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?s ?p ?o WHERE { ?s rdfs:label "電影"@zh ; ?p ?o }
Observation: 查無資料。
Thought:Thought: 我剛剛的查詢可能太過狹窄了，可能需要擴大查詢範圍。這次我可以嘗試查詢所有包含"電影"的實體。

Action: 本地 SPARQL 查詢工具
Action Input: PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?s ?p ?o WHERE { ?s ?p ?o . FILTER(contains(str(?s), "電影") || contains(str(?p), "電影") || contains(str(?o), "電影")) }
Observation: {'s': 'http://example.org/doc/金馬獎：一甲子的臺灣電影事業(183).json', 'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'o': 'http://xmlns.com/foaf/0.1/Document'}
{'s': 'http://example.org/doc/電影歎詠戰地、檔案銘刻戰役—金門古寧頭戰役(34).json', 'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'o': 'http://xmlns.com/foaf/0.1/Document'}
{'s': 'http://ex

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.3-70b-versatile` in organization `org_01j8hz1mc2eervtmepj9w29nxs` service tier `on_demand` on tokens per minute (TPM): Limit 12000, Requested 17279, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}